In [1]:
def form_initial_storage(chan_id, cust_addr, cust_pk, merch_addr, merch_pk, cust_bal0, merch_bal0, rev_lock, self_delay, pssig_addr):
    return '(Pair (Pair (Pair {chan_id} (Pair \"{cust_addr}\" 0)) (Pair (Pair {cust_bal0} \"{cust_pk}\") (Pair "0" \"{merch_addr}\"))) (Pair (Pair 0 (Pair {merch_bal0} \"{merch_pk}\")) (Pair (Pair \"{pssig_addr}\"  {rev_lock}) (Pair {self_delay} "awaitingFunding"))))'.format(chan_id=chan_id, cust_addr=cust_addr, cust_pk=cust_pk, merch_addr=merch_addr, merch_pk=merch_pk, cust_bal0=cust_bal0, merch_bal0=merch_bal0, self_delay=self_delay, rev_lock=rev_lock, pssig_addr = pssig_addr)

def form_closing_state(chan_id, cust_addr, merch_addr, cust_bal_mt, merch_bal_mt, new_rev_lock):

    return '(Pair (Pair \"{chan_id}\" (Pair \"{cust_addr}\" \"{merch_addr}\")) (Pair {cust_bal_mt} (Pair {merch_bal_mt} {rev_lock})))'.format(chan_id=chan_id, cust_addr=cust_addr, merch_addr=merch_addr, cust_bal_mt=cust_bal_mt, merch_bal_mt=merch_bal_mt, rev_lock=new_rev_lock)

def form_mutual_state(chan_id, cust_addr, merch_addr, cust_bal_mt, merch_bal_mt):

    return '(Pair (Pair \"{chan_id}\" \"{cust_addr}\") (Pair \"{merch_addr}\" (Pair {cust_bal_mt} {merch_bal_mt} )))'.format(chan_id=chan_id, cust_addr=cust_addr, merch_addr=merch_addr, cust_bal_mt=cust_bal_mt, merch_bal_mt=merch_bal_mt)

def read_json_file(json_file):
    f = open(json_file)
    s = f.read()
    f.close()
    return json.loads(s)

def add_hex_prefix(s):
    if s[:2] == "0x":
        return s
    return "0x" + s

def convert_to_little_endian(s):
    t = s
    if s[:2] == "0x":
        t = s[2:]
    return bytes.fromhex(t)[::-1].hex()

def get_cust_close_token(data):
    merch_pk = data.get("merch_pk")
    pubkey = {}
    for k,v in merch_pk.items():
        pubkey[k] = "0x" + str(v)
    m = data.get("message")
    channel_id = convert_to_little_endian(m["channel_id"])
    rev_lock = convert_to_little_endian(m["rev_lock"])
    message = [ 
        add_hex_prefix(channel_id), 
        add_hex_prefix(rev_lock),
        add_hex_prefix(int(m["cust_bal"]).to_bytes(32, 'little').hex()),
        add_hex_prefix(int(m["merch_bal"]).to_bytes(32, 'little').hex()),
    ]
    sig = data.get("signature")
    s1 = add_hex_prefix(sig.get("s1"))
    s2 = add_hex_prefix(sig.get("s2"))

    signature = [s1, s2]
    # print("Merch PK: %s" % pubkey)
    # print("Message: %s" % message)
    # print("Signature: %s" % signature)
    return (pubkey, message, signature)

def scenario_cust_close(pubkey, message, signature):
        burncap = "9"

        cust_addr = "tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx"
        cust_pk = "edpkuBknW28nW72KG6RoHtYW7p12T6GKc7nAbwYX5m8Wd9sDVC9yav"
        merch_addr = "tz1gjaF81ZRRvdzjobyfVNsAeSC6PScjfQwN"
        merch_pk = "edpktzNbDAUjUk697W7gYg2CRuBQjyPxbEg8dLccYYwKSKvkPvjtV9"
        contract_path = ""

        # Originate pssigs contract
        contract = "pssig.tz"
        pssig_name = "pssig_contract_d"
        initial_storage = '\"Unit\"'
        args = '--burn-cap {burncap}'.format(burncap=burncap)

        sender = "bootstrap1"

        cmd = 'tezos-client originate contract {contract_name} transferring 0 from {sender} running {contract} --init {initial_storage} {args}&'.format(contract_name=pssig_name, sender=sender, contract=contract, initial_storage=initial_storage, args=args)

        print("\npssig contract origination command:\n{cmd}\n\n".format(cmd=cmd))

        pssig_addr = "KT1WWEQH8ovHdErpwym4jP1NeAHaabjxTaG3"

        # Originate main zkChannel contract
        contract = contract_path + "zkchannel_main.tz"
        chan_id_fr, rev_lock_fr, cust_bal_fr, merch_bal_fr = message
        sig_s1, sig_s2 = signature

        zkchannel_name = "my_zkchannel_d"
        chan_id = chan_id_fr
        cust_bal = 20
        merch_bal = 10
        cust_bal_mt = cust_bal * 1000000
        merch_bal_mt = merch_bal * 1000000
        # Balance in mutez as bytes
        rev_lock0 = "0x1f98c84caf714d00ede5d23142bc166d84f8cd42adc18be22c3d47453853ea49"
        # self_delay = 86400    # seconds in 1 day (60*60*24)
        self_delay = 3

        # Originate zkchannel contract (without funding)
        initial_storage = '\'(Pair (Pair (Pair {chan_id} (Pair \"{cust_addr}\" 0)) (Pair (Pair {cust_bal_mt} \"{cust_pk}\") (Pair "0" \"{merch_addr}\"))) (Pair (Pair 0 (Pair {merch_bal_mt} \"{merch_pk}\")) (Pair (Pair \"{pssig_addr}\"  {rev_lock0}) (Pair {self_delay} "awaitingFunding"))))\''.format(chan_id=chan_id, cust_addr=cust_addr, cust_pk=cust_pk, merch_addr=merch_addr, merch_pk=merch_pk, cust_bal_mt=cust_bal_mt, merch_bal_mt=merch_bal_mt, self_delay=self_delay, rev_lock0=rev_lock0, pssig_addr = pssig_addr)

        args = '--burn-cap {burncap}'.format(burncap=burncap)
 
        cmd = 'tezos-client originate contract {contract_name} transferring 0 from {sender} running {contract} --init {initial_storage} {args}&'.format(contract_name=zkchannel_name, sender=sender, contract=contract, initial_storage=initial_storage, args=args)

        print("main zkchannel origination command:\n" + cmd + "\n")

        # Add funding 
        cmd = 'tezos-client transfer 20 from bootstrap1 to {contract_name} --burn-cap {burncap} --entrypoint addFunding&'.format(contract_name=zkchannel_name, burncap=burncap)
        print("add funding cust:\n" + cmd + "\n")
        cmd = 'tezos-client transfer 10 from bootstrap2 to {contract_name} --burn-cap {burncap} --entrypoint addFunding&'.format(contract_name=zkchannel_name, burncap=burncap)
        print("add funding merch:\n" + cmd + "\n")


        # Cust close entrypoint
        new_cust_bal_mt = 1 * 1000000
        new_merch_bal_mt = 29 * 1000000

        rev_lock = "0x90d774c7ce82fbe85a7bd34bf9cbb00689e1352e7bf6b54591ccd0d3fde9d729"

        # sample signature, merch-pk and g2 
        s1 = sig_s1 
        s2 = sig_s2 
        g2 = pubkey.get("g2") 
        merchPk0 = pubkey.get("Y0") 
        merchPk1 = pubkey.get("Y1") 
        merchPk2 = pubkey.get("Y2") 
        merchPk3 = pubkey.get("Y3") 
        merchPk4 = pubkey.get("X") 

        storage = '\'(Pair (Pair (Pair {new_cust_bal_mt} (Pair {cust_bal_fr} {g2})) (Pair (Pair {new_merch_bal_mt} {merch_bal_fr}) (Pair {merchPk0} {merchPk1}))) (Pair (Pair {merchPk2} (Pair {merchPk3} {merchPk4})) (Pair (Pair {rev_lock} {rev_lock_fr}) (Pair {s1} {s2}))))\''.format(s1=s1, s2=s2, g2=g2, merchPk0=merchPk0, merchPk1=merchPk1, merchPk2=merchPk2, merchPk3=merchPk3, merchPk4=merchPk4, rev_lock=rev_lock, new_cust_bal_mt=new_cust_bal_mt, new_merch_bal_mt=new_merch_bal_mt, cust_bal_fr=cust_bal_fr, merch_bal_fr=merch_bal_fr, rev_lock_fr=rev_lock_fr)

        cmd = 'tezos-client transfer 0 from {sender} to {contract_name} --entrypoint custClose --burn-cap {burncap} --arg {storage}&'.format(contract_name=zkchannel_name, sender=sender, contract=contract, burncap=burncap, storage=storage)

        print("custClose call command:\n" + cmd)

In [2]:
data = {"merch_pk":{"Y1":"11773bb86f7d8a50c5ef88ef515ed4ef3313278c67ce800196ee13d790a3f88a0ea094f9a67422fee73ecc5199ac109a1839d74d054d0cfe8467e773a6e0f2c81848c562035d4132b5f1e1fec343a77c1490c113316b92c2a7314ca33aa4932608df39e38326c3069c53c9fe877794f01e279621599123cbad1025fe87aedf763455ea457226d2f9a8d53d307d65e14510d1ad7031dae45c1e51b971c4964b549572b44edf97493b231a3d83c9474da416f53a8f036b07110a5577830987b76a","X":"08f55c27b65bfe1cb0fc182a73c799f4955e13f3d48387422bffb81d7baf811098b19685558caa0c8f2834ef32d1a3910a55bfb3362d4545b8e8bf5e155af782930f120e295d95aaa9c144f4a37bb32363f7738d43c6c6aceef1e210187813c3164e73cc036b738436b7621e414bcebab60c3e6f6700be7fda35303c8b9e4ccc168db7f65e66739f37109760eb848fdf0a71d5bacacf1518e17948d266effe04b97a7b21336e33a68d0eba07a51c5360231df2dc1efba4f8a5a33296b3e723c7","Y0":"083930b7626dbe92878de408eea223a6f7a183a6d18c6442ab801d558639322d7f37bd27073532e551d5e31a5b48a7fb055ac29b37c1d1f9092a57a476758e0be023cb43dd368d6dc30223a648324ce93d051469ffbcfe20a038bced8dbff606174e12e8a91c6a2e511c7b9aaae76a66191bb960b2571def75e231d3f6211aa98004cbb4e70ad9668fd8d386957479cb040bfc6519d793a8db488a0c604471deed80bb942c5730b8a94f54e898cbfc33ccfac15268b808c806b84111a5af921e","g2":"0990a9b13921c01f387c2725ee5c4c47a56825136dc85f9b55f5244dfe45be8aa610a7358905fc6a9ab848c850fb45ac097fc1f3b2aa09b660b8a7069d2db75f21fd9a924d15a855d93f8f4540f7626a4df560e9c12e215db86fdebb9b6bf8b5086129163c9e6071207ff43743012d37893e2b52fb75ff63dc35fefb7266af9883f4566ae2762862ae7084c4fd39381d185946e1c1280d87d3256673b9bb680cefe926e3e5ae194d732f762c62725ad13d3cecaefa1a75a724b58af3f1c9bd18","Y2":"1622c35b21afdd2223db3baa7ae35a6b6981951c711cf851dc1935e59a49f581cf7e33b8a5af1743bfe1e5c029cc12780021ab98bbb27eaf0031e3c7aebb8e496c7b4f0d6b210e8523b63b64be9105eeda7caf8ab34100c63e124fc647acd5c311feaeecc7fab921cb38b81d382398976e37b9a883d1b79c9c100849b5df3576af38870629269bb9342ea1d02cab3f5b018057f5c59e8f7a391348f34f31e7c3b1c375d6a03e065f29cd7ac3db28808ddf0f4367c3614dc995104f9103317f70","Y4":"0c2214399b784ea3cce84b5b0e172e875fde889e301d03380a27e9054c70aeee340977303f721c7f12ae9227d695372d096e73560e295df569fa520de7a7c650f3c32f3b92cecbdeff541f2891e9df6da3e1719e842fe757d8d68bda0cac8ef214818222001fa1dacca3a68045e70a385ad87a97b9471f3281a0bad10b992b136b4ac9498f0f84b4591ed1d0ddbbd80a0c23d1c4af8c9f82127f7d338243e558aedb546f6be739b7def355e5b37cbaa0403a5dd89cd776ff6e52e4ce1864c033","Y3":"18be8b41978a2aa56e77a46246438c20edb5145dcc53c8430ebe809899a0ab1db74c29038d10fb72707d6630b00d00bd048c0508011220765f4be805fc4d600562ce28b9aa6174880fd83d1bec0848df8b92d16906341d374156dbf0bf3b0111049e562db0222d419a2e3e11430b1acad3466506499a1f2f4872a03f7dfb97b92466d9723b411309b2cf5bbab043ae840d1f67d445ddfb776db6c524c14467faddd7fa7adfabbbb5edbb60631c5c52d7283802e9073d9b345e12fdde2517796e"}, "message":{"merch_bal":"10000200","channel_id":"0x5dffcd6f357f20a862f927fb7919e28ff3214977ed0a38232e907c4ab73691f0","cust_bal":"19999800","rev_lock":"0x5c2e79664c13a8fc34b93e8882b9abf583208c6bc8cd2ad11e68d1232e88e68f"}, "signature":{"s1":"1713091665473295e5ef2f6994c2a20aef21b38685558a4f938ed2cbdda411edf584bcfab31a3d55634ee08dfd5376bc19abad5e5e2e8e64d21b13ecb810601b49ae708fc383609068bee51e69db56af9030693a645fc0ef7df2024a4cde456d","s2":"0ef3081cbce139afe9f14da76b5e337b5a097c20bb62226f97086d851dc66bf72e85258eee96297fb9f6f955c74d65b00f9587346e58af360f8c7bb6e7d729c0fc99a8976d7ccd96c0c662ec974e05e44fc6be7ca3ef540998839b8ffdad2128"}}

In [3]:
merch_pk = data.get("merch_pk")
pubkey = {}
for k,v in merch_pk.items():
    pubkey[k] = "0x" + str(v)
m = data.get("message")
channel_id = convert_to_little_endian(m["channel_id"])
rev_lock = convert_to_little_endian(m["rev_lock"])
message = [ 
    add_hex_prefix(channel_id), 
    add_hex_prefix(rev_lock),
    add_hex_prefix(int(m["cust_bal"]).to_bytes(32, 'little').hex()),
    add_hex_prefix(int(m["merch_bal"]).to_bytes(32, 'little').hex()),
]
sig = data.get("signature")
s1 = add_hex_prefix(sig.get("s1"))
s2 = add_hex_prefix(sig.get("s2"))

signature = [s1, s2]

In [4]:
scenario_cust_close(merch_pk, message, signature)


pssig contract origination command:
tezos-client originate contract pssig_contract_d transferring 0 from bootstrap1 running pssig.tz --init "Unit" --burn-cap 9&


main zkchannel origination command:
tezos-client originate contract my_zkchannel_d transferring 0 from bootstrap1 running zkchannel_main.tz --init '(Pair (Pair (Pair 0xf09136b74a7c902e23380aed774921f38fe21979fb27f962a8207f356fcdff5d (Pair "tz1KqTpEZ7Yob7QbPE4Hy4Wo8fHG8LhKxZSx" 0)) (Pair (Pair 20000000 "edpkuBknW28nW72KG6RoHtYW7p12T6GKc7nAbwYX5m8Wd9sDVC9yav") (Pair "0" "tz1gjaF81ZRRvdzjobyfVNsAeSC6PScjfQwN"))) (Pair (Pair 0 (Pair 10000000 "edpktzNbDAUjUk697W7gYg2CRuBQjyPxbEg8dLccYYwKSKvkPvjtV9")) (Pair (Pair "KT1WWEQH8ovHdErpwym4jP1NeAHaabjxTaG3"  0x1f98c84caf714d00ede5d23142bc166d84f8cd42adc18be22c3d47453853ea49) (Pair 3 "awaitingFunding"))))' --burn-cap 9&

add funding cust:
tezos-client transfer 20 from bootstrap1 to my_zkchannel_d --burn-cap 9 --entrypoint addFunding&

add funding merch:
tezos-client transfer 10 from boot